# Recommendation Using TF-IDF weighted Words Embedding

**STEP** <br>
1. Create TF-IDF 
2. Convert a tf-idf dictionary with word as key, idf as a value
3. Get TF-IDF features
4. Combine pretrained words embedding with TF-IDF
5. Calculate Cosine Similarity 
6. Recommend Law 

### Read ckiptagger & Dataframe

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
import datetime
import pickle
import jieba

path = "./ckip/data"
ws = WS(path)

df = pd.read_csv('../data/courses.csv').fillna('')
# Replace '@' with ' ' in original dataframe
#df.token = df.token.apply(lambda text: str(text).replace('@',' '))

C:\Users\Vincent\AppData\Local\Programs\Python\Python310\lib\site-packages\ckiptagger\model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


### Import Words Dictionary

In [2]:
# dictionary
# dict_path = './dictionary'
# legal_name_file = dict_path + '/name_of_legal.txt'
# word_file = dict_path + '/oth_words.txt'
# split_rule_kw_file = dict_path + '/split_rule_words.txt'

# with open(legal_name_file, 'r', encoding='big5') as k1, open(word_file, 'r', encoding='big5') as k2:
#     k = k1.read().split('\n') + k2.read().split('\n')
#     word_to_weight = dict([(_, 1) for _ in k])
#word_dict = construct_dictionary(word_to_weight)

### Read Pretrained Words Embedding
詞向量訓練文本來源為中文維基百科，全部的訓練文本可於[此](https://dumps.wikimedia.org/zhwiki/latest/zhwiki-latest-pages-articles.xml.bz2)下載最新版的中文維基百科。<br>
維基百科2014（總詞彙數：655K，400維詞向量，下載大小為2.5G）<br>
來源：[元智大學自然語言處理實驗室](http://nlp.innobic.yzu.edu.tw/demo/word-embedding.html)

In [3]:
# # https://ithelp.ithome.com.tw/articles/10194633
# embeddings = {}
# f = open('wiki.zh.vector', encoding = 'utf8') 
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings[word] = coefs
# f.close()

In [4]:
token_list = np.load('./ckip/data/embedding_word/token_list.npy')
vector_list = np.load('./ckip/data/embedding_word/vector_list.npy')

embeddings = dict(zip(token_list, vector_list))
d = vector_list.shape[1]

In [5]:
course_df_original = pd.read_csv('../data/courses.csv').fillna('')
course_df = course_df_original.drop(['course_id', 'teacher_id', 'course_published_at_local', 'course_published_at_local', 'course_price'], axis=1)
course_df['description'] = course_df['description'].replace('([\<]).*?([\>])','',regex=True)
course_df = course_df.replace('\n', '',regex=True)
course_df = course_df.replace('[\d_]', '',regex=True).astype(str)
course_df.insert(0, 'course_id', course_df_original['course_id'])
course_df[:2]
info = []
for i in range(course_df.shape[0]):
    t = course_df.loc[i, :].values.flatten().tolist()
    info.append([t[0], ' '.join(t[:])])
info = pd.DataFrame(info, columns = ['course_id', 'text'])
info[:5]

,course_id,text
0,61888e868f154b000781b45a,61888e868f154b000781b45a 少女人妻華麗變身：七大妝容七彩的夢幻樂園 ...
1,54d5a117065a7e0e00725ac0,54d5a117065a7e0e00725ac0 幾何圖形分割 X 色塊組合 從學生時代開始...
2,54d5d9952246e60a009ec571,54d5d9952246e60a009ec571 數位拼貼的手感 自由工作者，致力於品牌視覺...
3,54d7148a2246e60a009ec588,54d7148a2246e60a009ec588 Line 的貼圖自己動手做！ 我是Dann...
4,5513e92b38239d10005778e1,5513e92b38239d10005778e1 為申請學校或工作寫好英文自傳 在北美長大，...


### Tf-idf for Tokenized Text in Dataframe

In [6]:
# TF-IDF Model
tfidf_ml = TfidfVectorizer(tokenizer=jieba.lcut)
tfidf_ml.fit(info.text)

# TF-IDF Dicitonary
dictionary = dict(zip(tfidf_ml.get_feature_names(), list(tfidf_ml.idf_)))

# feature name
tfidf_feature = tfidf_ml.get_feature_names()
'|'.join(tfidf_feature[5000:5050])

C:\Users\Vincent\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Vincent\AppData\Local\Temp\jieba.cache
Loading model cost 0.624 seconds.
Prefix dict has been built successfully.
C:\Users\Vincent\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


'oo|ooad|ooo|ooxx|opec|open|openapi|opencl|opencv|opendata|opener|openers|openframeworks|opengl|opening|openmindenjoylife|opensea|openstreetmap|openzeppelin|operability|operating|operation|operations|opi|opp|oprah|ops|optical|opticalflares|optimization|optimize|or|oracle|orbit|ord|order|oren|oreo|org|organise|organization|organize|oriented|origin|original|orm|os|osaka|oscar|oscow'

### Newly Entered Text Preprocess function
- Remove Punctuation
- Remove Spaces
- Sentence Segment
- turn into list

In [7]:
def Preprocess(text):
    rule = re.compile(r'[^a-zA-Z0-9\u4e00-\u9fa5]')
    text = rule.sub(' ',str(text))
    text = re.sub(' +', '',text)
    #text = ws([text],sentence_segmentation=True, recommend_dictionary=word_dict)
    text = ws([text],sentence_segmentation=True)
    text = [x for l in text for x in l]
    return(text)

### Calculate TF-IDF Weighted Word Embedding

In [8]:
starttime = datetime.datetime.now()

# TF-IDF weighted Word2Vec
tfidf_text_vect = [] # tfidf-w2v is stored in this list
row = 0

for text in info.text.apply(lambda text: text.split()):
    text_vect = np.zeros(300)
    weight_sum = 0
    for word in text:
        if word in embeddings.keys() and word in tfidf_feature:
            vec = embeddings[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tfidf_text_vect.append(text_vect)
    row += 1

# calculate running time
endtime = datetime.datetime.now()
print("建立模型時間: ",endtime - starttime)

建立模型時間:  0:00:28.400724


### Law Recommendation Function
輸入內文 --> 跑出推薦的前十個相近內文對應的法律

In [34]:
def recommend_law(text, tfidf_text_vect = tfidf_text_vect):
    text = Preprocess(text)
    #print(text)
    text_vect = np.zeros(300) # w2v size
    weight_sum = 0
    for word in text:
        if word in embeddings.keys() and word in tfidf_feature:
            vec = embeddings[word]
            tf_idf = dictionary[word]*(text.count(word)/len(text))
            text_vect += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        text_vect /= weight_sum
    tmp_vect = [*tfidf_text_vect,text_vect]
    new_cos_sim = cosine_similarity(tmp_vect, tmp_vect)
    sim_score = np.sort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:51]
    tmp_top_10_law = info[['text']].iloc[np.argsort(new_cos_sim[new_cos_sim.shape[0]-1])[::-1][1:51]]
    tmp_top_10_law['similarity_score'] = [round(score*100,1) for score in sim_score]
    return tmp_top_10_law

In [10]:
#tfidf_feature

### Try an Example
輸入內容便可以推薦出適合的法律<br>
(這邊列出的CE_Comment純粹是用來比對「輸入的內容」跟「原本內文」是否真的相近)

In [35]:
###### starttime = datetime.datetime.now()

newtext = '職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投資觀念,行銷_數位行銷,藝術_角色設計,藝術_繪畫與插畫,職場技能_個人品牌經營'
result = recommend_law(newtext)

# calculate running time
endtime = datetime.datetime.now()
print("搜尋推薦時間: ",endtime - starttime)
result


搜尋推薦時間:  0:18:35.152785


,text,similarity_score
123,58abfb2f5c4e6507007c2b46 AE/MG || 進階動畫特效 Vol. ...,75.9
56,56fbd6112ba01d09002bae64 畫，故事 //手繪深度圖像創作 對大多是的...,72.9
40,568552e0e8ff9b20003cf518 角色設計人體骨架結構 遊戲美術工作者，參與...,72.6
6,551a6be023774e0a001eb20c 療癒感手繪插畫食譜 很難明確的表達自己有多...,72.6
87,5737f2acfad4ef0a006b6912 邊走邊畫，帶著手帳旅行去！ 插畫家與旅行作...,72.6
19,5591501e6dec460f00111314 動物插畫 吳振宇（Johnny） 年出生於...,72.6
44,563ab301423bdd0a00103dd7 一張速寫，一個生活故事 我喜歡畫畫、寫作，...,72.6
34,56499b1542ee8d100033a0fa 旅行中手繪建築插畫，你也可以！ 你要熱愛自...,72.6
59,56d68e0b182fba0e00c6d9d7 超濃郁！美味的食物插畫 藝術 繪畫與插畫...,72.6
8,54f1268f4ec3c809002e4a29 不一樣的西洋美術史 筱筑。artist。繪...,72.6


In [36]:
users = pd.read_csv('../data/users.csv').fillna('')
users

,user_id,gender,occupation_titles,interests,recreation_names
0,54ccaa73a784960a00948687,female,,"職場技能_創業,藝術_電腦繪圖,設計_介面設計,設計_動態設計,設計_平面設計,投資理財_投...",
1,54dca4456d7d350900e86bae,male,,"設計_動態設計,設計_平面設計,設計_應用設計,程式_程式入門,程式_程式語言,藝術_角色設...",
2,54e421bac5c9c00900cd8d47,female,,"設計_平面設計,職場技能_資料彙整,藝術_繪畫與插畫,行銷_數位行銷,職場技能_文書處理,職...",
3,54e961d4c5c9c00900cd8d84,other,金融業,"投資理財_理財,攝影_影像創作,投資理財_投資觀念,藝術_更多藝術,音樂_樂器,投資理財_金...",
4,54e9b744c5c9c00900cd8d8a,other,"資訊科技,法律、社會及文化專業,非營利組織","程式_網頁前端,投資理財_理財,投資理財_投資觀念,程式_程式語言,設計_設計理論,投資理財...","政治經濟,社會服務,舞台劇,電影"
...,...,...,...,...,...
130561,62e09de8fc3d3500060d4211,female,,"語言_英文,設計_介面設計,設計_網頁設計,設計_設計理論,程式_軟體程式開發與維護,行銷_...",
130562,62f0823a8c4414000667c592,,,,
130563,631b86242145060007efc7dd,,,,
130564,6331648104ed0f000610dfd2,male,公務人員,"投資理財_理財,攝影_影像創作,攝影_後製剪輯,攝影_商業攝影,投資理財_投資觀念","旅行旅遊,運動健身,金融理財,電影"


In [47]:
user_dict = {}
for user, title, interest, name in zip(users['user_id'], users['occupation_titles'],users['interests'],users['recreation_names']):
    user_dict[user] = ' '.join(title.split(',')) + ' ' + ' '.join(interest.split(',')) + ' ' + ' '.join(name.split(','))
list(user_dict.items())[:10]

[('54ccaa73a784960a00948687',
  ' 職場技能_創業 藝術_電腦繪圖 設計_介面設計 設計_動態設計 設計_平面設計 投資理財_投資觀念 行銷_數位行銷 藝術_角色設計 藝術_繪畫與插畫 職場技能_個人品牌經營 '),
 ('54dca4456d7d350900e86bae',
  ' 設計_動態設計 設計_平面設計 設計_應用設計 程式_程式入門 程式_程式語言 藝術_角色設計 設計_介面設計 程式_網頁前端 藝術_電腦繪圖 音樂_音樂創作 藝術_繪畫與插畫 '),
 ('54e421bac5c9c00900cd8d47',
  ' 設計_平面設計 職場技能_資料彙整 藝術_繪畫與插畫 行銷_數位行銷 職場技能_文書處理 職場技能_職場溝通 '),
 ('54e961d4c5c9c00900cd8d84',
  '金融業 投資理財_理財 攝影_影像創作 投資理財_投資觀念 藝術_更多藝術 音樂_樂器 投資理財_金融商品 音樂_音樂創作 '),
 ('54e9b744c5c9c00900cd8d8a',
  '資訊科技 法律、社會及文化專業 非營利組織 程式_網頁前端 投資理財_理財 投資理財_投資觀念 程式_程式語言 設計_設計理論 投資理財_金融商品 政治經濟 社會服務 舞台劇 電影'),
 ('54eac49bc5c9c00900cd8d95', ' 行銷_文案 投資理財_理財 投資理財_投資觀念 行銷_數位行銷 '),
 ('54ef3499b839040f0001607d',
  ' 攝影_商業攝影 設計_平面設計 藝術_電腦繪圖 攝影_影像創作 藝術_角色設計 藝術_繪畫與插畫 插畫 桌遊 棋類遊戲 素描 電影 電玩'),
 ('54ef34e2b839040f00016086', ' 職場技能_職場溝通 職場技能_資料彙整 行銷_數位行銷 職場技能_文書處理 '),
 ('54ef354db839040f0001608b',
  '藝文設計 自由業 音樂_人聲 藝術_電腦繪圖 音樂_音樂創作 程式_遊戲開發 藝術_字體設計 狗派 運動健身 電玩 電腦繪圖'),
 ('54ef3574b839040f0001608f',
  '藝文設計 出版業 自由業 職場技能_創業 職場技能_求職 職場技能_效率提升 程式_程式入門 職場

In [52]:
test_unseen = pd.read_csv('../data/test_unseen.csv')
with open('./submit_unseen_course.csv', 'w') as f:
    f.write('user_id,course_id\n')
    for i, user in  enumerate(test_unseen['user_id'][:]):
        t = []
        if(i % 500 == 0):
            print(i)
        for course in recommend_law(user_dict[user])['text']:
            
            t.append(course[:24])
        f.write('{},{}\n'.format(user, ' '.join(t)))

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
